In [ ]:
! python -m pip install --upgrade pip 

In [77]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Neo4jSparkApp") \
    .config("spark.jars.packages","org.neo4j:neo4j-connector-apache-spark_2.12:5.3.2_for_spark_3") \
    .config("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.0-s_2.12") \
    .config("neo4j.url", "bolt://localhost:7687") \
    .config("neo4j.authentication.basic.username", "neo4j") \
    .config("neo4j.authentication.basic.password", "krishna1") \
    .config("neo4j.database", "neo4j") \
    .getOrCreate()

In [78]:
df=(
    spark.read.format("org.neo4j.spark.DataSource")
    .option("labels","Paper")
    .load()
    .show()
    )

+----+--------+----------+
|<id>|<labels>|        id|
+----+--------+----------+
|   0| [Paper]|2150881840|
|   1| [Paper]|1971405342|
|   2| [Paper]|2160484294|
|   3| [Paper]|2147271487|
|   4| [Paper]|1967380282|
|   5| [Paper]|2048748336|
|   6| [Paper]|2805510628|
|   7| [Paper]|2407716750|
|   8| [Paper]|2150203549|
|   9| [Paper]|2971329390|
|  10| [Paper]|2052589210|
|  11| [Paper]|1993983849|
|  12| [Paper]|3022911403|
|  13| [Paper]|2912205033|
|  14| [Paper]|2964084596|
|  15| [Paper]|2963132284|
|  16| [Paper]|2167322980|
|  17| [Paper]|2141829656|
|  18| [Paper]|2147345946|
|  19| [Paper]|2125859743|
+----+--------+----------+
only showing top 20 rows



In [79]:
df=(
    spark.read.format("org.neo4j.spark.DataSource")
    .option("labels","CITES")
    .load()
    .show()
    )

+----+--------+
|<id>|<labels>|
+----+--------+
+----+--------+



In [80]:
nodes_df = spark.read.format("org.neo4j.spark.DataSource") \
    .option("labels", ":Paper") \
    .load()

relationships_df = spark.read.format("org.neo4j.spark.DataSource") \
    .option("relationship", "CITES") \
    .option("relationship.source.labels", ":Paper") \
    .option("relationship.target.labels", ":Paper") \
    .load()


In [81]:
! pip install graphframes

In [82]:
nodes_df.printSchema()


root
 |-- <id>: long (nullable = false)
 |-- <labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)



In [83]:
relationships_df.printSchema()
relationships_df.show(5, truncate=False)


root
 |-- <rel.id>: long (nullable = false)
 |-- <rel.type>: string (nullable = false)
 |-- <source.id>: long (nullable = false)
 |-- <source.labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- source.id: string (nullable = true)
 |-- <target.id>: long (nullable = false)
 |-- <target.labels>: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- target.id: string (nullable = true)



+-------------------+----------+-----------+---------------+----------+-----------+---------------+----------+
|<rel.id>           |<rel.type>|<source.id>|<source.labels>|source.id |<target.id>|<target.labels>|target.id |
+-------------------+----------+-----------+---------------+----------+-----------+---------------+----------+
|1152921504606890866|CITES     |43890      |[Paper]        |2101923777|1          |[Paper]        |1971405342|
|1157425104234221918|CITES     |4446       |[Paper]        |2117928809|2          |[Paper]        |2160484294|
|1152921504606884718|CITES     |37742      |[Paper]        |2133936558|2          |[Paper]        |2160484294|
|1152921504606846978|CITES     |2          |[Paper]        |2160484294|16         |[Paper]        |2167322980|
|1152921504606851302|CITES     |4326       |[Paper]        |2166142148|16         |[Paper]        |2167322980|
+-------------------+----------+-----------+---------------+----------+-----------+---------------+----------+
o

In [84]:
! pip install graphframes


In [85]:
simrank_results = {0.7: {"node": 2982615777, "similar_nodes": {1556418098: [{"node": 1995529998, "similarity": 0.37}, {"node": 121193193, "similarity": 0.065}, {"node": 2518062904, "similarity": 0.065}, {"node": 2118756286, "similarity": 0.0353125}, {"node": 2131415946, "similarity": 0.01765625}, {"node": 1762028752, "similarity": 0.004265625}]}}, 0.8: {"node": 2982615777, "similar_nodes": {1556418098: [{"node": 1995529998, "similarity": 0.43}, {"node": 121193193, "similarity": 0.09}, {"node": 2131660762, "similarity": 0.09}, {"node": 2118756286, "similarity": 0.045}, {"node": 1491797991, "similarity": 0.045}, {"node": 2015372742, "similarity": 0.0225}, {"node": 2131415946, "similarity": 0.0225}, {"node": 1762028752, "similarity": 0.005625}]}}, 0.9: {"node": 2982615777, "similar_nodes": {1556418098: [{"node": 1995529998, "similarity": 0.47}, {"node": 2122991029, "similarity": 0.16}, {"node": 121193193, "similarity": 0.11}, {"node": 2518062904, "similarity": 0.11}, {"node": 2131660762, "similarity": 0.11}, {"node": 2118756286, "similarity": 0.055}, {"node": 2015372742, "similarity": 0.0275}, {"node": 1762028752, "similarity": 0.006953125}]}}}

In [ ]:
from neo4j import GraphDatabase
import networkx as nx
from tqdm import tqdm
# import neo4j_creds
import pandas as pd
from pyspark.sql import SparkSession

if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("Neo4jGraph") \
        .getOrCreate()

    driver = GraphDatabase.driver(
        "bolt://localhost:7687", auth=("neo4j", "krishna1")
    )

    with driver.session() as session:
        result = session.run(
            """
            MATCH (a:Paper)-[:CITES]->(b:Paper) 
            RETURN a.id AS source, b.id AS target
            """
        )

        edges = [
            (record["source"], record["target"])
            for record in tqdm(result, desc="Fetching edges")
        ]
        

        edges_df = spark.createDataFrame(edges, ["source", "target"])
    
    driver.close()

    graph = nx.DiGraph()
    for row in tqdm(edges_df.collect(), desc="Adding edges to graph"):
        graph.add_edge(row["source"], row["target"])

    nx.write_edgelist(graph, "graph.edgelist")
    
    print("Graph processing complete!")
    spark.stop()


In [ ]:
import networkx as nx
from collections import defaultdict

def efficient_simrank(graph, target_nodes, decay_factor=0.8, max_iterations=6, top_results=10):
    target_nodes = [str(node) for node in target_nodes]
    
    incoming_neighbors = {node: set(graph.predecessors(node)) for node in graph.nodes()}

    def calculate_pair_similarity(node_u, node_v, current_iteration):
        if node_u == node_v:
            return 1.0
        
        if current_iteration == 0:
            return 0.0
        
        neighbors_u = incoming_neighbors[node_u]
        neighbors_v = incoming_neighbors[node_v]
        
        if not neighbors_u or not neighbors_v:
            return 0.0
        
        total_similarity = 0.0
        i = 0
        while i < len(neighbors_u):
            j = 0
            while j < len(neighbors_v):
                total_similarity += calculate_pair_similarity(neighbors_u[i], neighbors_v[j], current_iteration - 1)
                j += 1
            i += 1
        
        return decay_factor * total_similarity / (len(neighbors_u) * len(neighbors_v))
    
    similarity_results = {}
    
    i = 0
    while i < len(target_nodes):
        node = target_nodes[i]
        if node not in graph:
            i += 1
            continue
            
        node_similarities = []
        j = 0
        while j < len(graph.nodes()):
            other_node = str(list(graph.nodes())[j])
            if other_node != node:
                similarity_score = calculate_pair_similarity(node, other_node, max_iterations)
                if similarity_score > 0:
                    node_similarities.append((other_node, similarity_score))
            j += 1
        
        node_similarities.sort(key=lambda x: x[1], reverse=True)
        similarity_results[node] = node_similarities[:top_results]
        i += 1
    
    return similarity_results

target_nodes = ['2982615777', '1556418098']
decay_factors = [0.7, 0.8, 0.9]

similarity_output = {}
k = 0
while k < len(decay_factors):
    similarity_output[decay_factors[k]] = efficient_simrank(graph, target_nodes, decay_factor=decay_factors[k])
    k += 1


In [ ]:
import pprint

pprint.pprint(simrank_results, width=100)

{0.7: {'node': 2982615777,
       'similar_nodes': {1556418098: [{'node': 1995529998, 'similarity': 0.37},
                                      {'node': 121193193, 'similarity': 0.065},
                                      {'node': 2518062904, 'similarity': 0.065},
                                      {'node': 2118756286, 'similarity': 0.0353125},
                                      {'node': 2131415946, 'similarity': 0.01765625},
                                      {'node': 1762028752, 'similarity': 0.004265625}]}},
 0.8: {'node': 2982615777,
       'similar_nodes': {1556418098: [{'node': 1995529998, 'similarity': 0.43},
                                      {'node': 121193193, 'similarity': 0.09},
                                      {'node': 2131660762, 'similarity': 0.09},
                                      {'node': 2118756286, 'similarity': 0.045},
                                      {'node': 1491797991, 'similarity': 0.045},
                                      {'nod